# How accurately can we recover the SFR threshold age?
---
In this notebook, we will see how accurately we can recover the SFR threshold age via MC simulations with ppxf, applied to a variety of different SFHs.
We will save the figures for each individual SFH to a .pdf for later reference.
